In [109]:
import pandas as pd
import numpy as np


In [110]:
data = pd.read_csv('data/dfGoogle.csv')
data = data.drop('ID', axis=1)
data

,Event,Tie,Chin,Smile,Hat,Font,Forehead,Gesture,Joy,Person,...,domingo,quarta-feira,quinta-feira,segunda-feira,sexta-feira,sábado,terça-feira,Perfil,Hashtag,Texto
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0,0,1,0,0,0,0,1,1,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,1
2,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0,0,1,0,0,0,0,1,1,0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0,0,1,0,0,0,0,1,1,0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,1,1,1
565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,0,1,1
566,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,1,1
567,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,1,0,0,1,1


In [111]:
x_data = data.drop('Curtidas', axis=1).values
y_data = data['Curtidas'].values

In [112]:
x_data

array([[1., 0., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 1., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [113]:
y_data

array([  24400,   11700,    9163,    3485,   22100,   33600,  115700,
         34600,    3958,   11900,   47500,    8068,   22400,   41200,
         12000,  394000,    4634,    5384,    8871,   13700,   25100,
        198100,   10200,   24000,    6123,    4641,   18500,    2368,
         13100,    4149,   18100,    4147,   45200,    6987,   17200,
          4583,    5284,    6533,  146400,   33900,   25800,   58600,
         12100,   28800,    8903,    3919,   20600,   38700,    7606,
          2963,    4977,    5682,   10200,   60500,    9316,    6988,
          9152,    6434,    9393,    5828,   61500,   39500,   16000,
          8219,   52500,   13500,    4833,   11300,    4895,    2647,
          3259,    2937,    3969,  258200,   28400,    4373,   16500,
         12100,   52300,   13600,    4103,   15600,   44500,    4337,
          4326,  175600,  140600,  390100,   16000,   18900,  439500,
         16800,  290000,    3974,   25600,   23700,    9521,  302300,
         12200,    6

## Separando dados de treino

In [114]:
from sklearn.model_selection import train_test_split 

In [115]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

In [116]:
x_train.shape, y_train.shape

((512, 21), (512,))

In [117]:
x_test.shape, y_test.shape

((57, 21), (57,))

In [118]:
y_test

array([216400,   3259,   3638,  61200,  52500,   4807, 264000,   4149,
        15600,  21400,  41900,   4228, 109500,  18300,  10700,   4337,
        29800, 104900,   5628,  18100, 258200,   9982,  51600,   9596,
        12300,  16500,  74700,  61300,   6313,   3969, 332800,  15400,
        16800,   3106, 418400,   6988,  47500,   8911, 115700, 439500,
        15700,  39200, 409400, 107500,  20400, 169400,   5981,  36600,
        15400,  14700,  53900,  18100,  62700,  34900,  30300,  32400,
         5619], dtype=int64)

In [121]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression(
  n_jobs=1,
)
lr.fit(x_train, y_train)
previ = lr.predict(x_test)
lr.score(x_test, y_test)

-0.46399460553609106

In [123]:
previ.shape

(57,)

In [126]:
y_test.shape

(57,)

In [131]:
from sklearn.tree import DecisionTreeRegressor
dtr3 = DecisionTreeRegressor(max_depth=2)
dtr3.fit(x_train, y_train)
dtr3.score(x_test, y_test)

-0.12524371676959145

array([148297.65481261,  79248.80183438,  12218.85930092, 172822.93353861,
        25594.59448548, 215511.3564236 , 124147.97004532, 143922.60551281,
        40027.92004047,  82765.81494989,  18423.45980793,  90920.49838366,
       215471.11506079,  91240.44040833,  94351.87888387, 145778.39667665,
       116536.73154771,  85258.30042839,  91425.16366461, 172639.8879217 ,
        74464.07408999, 109648.22196764, 148574.53091856,  34705.17818309,
       199088.67069085, 112346.13838952,  10993.62537065, 149111.99706055,
         5151.37071149,  90294.43182647, 150523.01516744,  74211.8448457 ,
       246896.5842924 ,  -4181.99476791,  68964.99857876, 160448.57283022,
       126125.44221   , 175232.11168035,  78788.21545215,  37209.01525102,
        58120.81653404,  -8119.15045985, 178733.2463645 ,  16732.5046421 ,
       117339.071671  ,  49668.96318001,  38248.40740136,  98360.02317766,
       180534.85861069, 224073.86223208, 134340.61369575, 193910.86608348,
        69476.35721753, 1